In [34]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import re
import seaborn as sns
import statsmodels.stats.multitest
import sys
import urllib3
import json

import CPTAC.Endometrial as CPTAC

## Edit the following to configure the notebook

In [35]:
'''Edit Below'''

gene = "PIK3CA" #The gene you want to examine

number_of_interacting_proteins_to_examine = 30 #The number of interacting proteins you want to look at

hotspot = True #(True or False) Whether you want to examine the effect of hotspot mutations in this gene

proteomics = True #(True or False) Whether you want to see results of proteomic data

phosphoproteomics  = True #(True or False) Whether you want to see results of phosphoproteomic data

transcriptomics = True #(True or False) Whether you want to see results of transcriptomic data

subtype = True #(True or False) Whether you want to examine impact of subtype

dataset = 'Endometrial' #options are Endometrial, Ovarian, etc.

## Access STRING database to compile list of interacting proteins

In [36]:
'''Use urllib3 to access the string database api, gather list of interacting proteins'''
urllib3.disable_warnings()
string_api_url = "https://string-db.org/api"
output_format = "json"
method = "network"

'''Use the specified gene and homo sapiens species code'''
my_gene = [gene]
species = "9606"

'''format the api request to collect the appropriate information'''
request_url = string_api_url + "/" + output_format + "/" + method + "?"
request_url += "identifiers=%s" % "%0d".join(my_gene)
request_url += "&" + "species=" + species
request_url += "&" + "limit=" + str(number_of_interacting_proteins_to_examine)

'''Send a request to the API, print the response status'''
try:
    http = urllib3.PoolManager()
    response = http.request('GET',request_url)
    print("Accessing String database, response status: ", response.status)
except urllib3.HTTPError as err:
    error_message = err.read()
    print(error_message)
    sys.exit()

interacting_proteins = []
if response.status == 200: 
    '''Get the data from the API's response'''
    data = response.data
    y = json.loads(data)

    '''Make a list of the resulting interacting proteins'''
    for entry in y:
        if entry["preferredName_A"] not in interacting_proteins:
            interacting_proteins.append(entry["preferredName_A"])
        if entry["preferredName_B"] not in interacting_proteins:
            interacting_proteins.append(entry["preferredName_B"])

    print("\nTop " + str(number_of_interacting_proteins_to_examine) + " " + gene + " interacting proteins: \n")
    for ip in interacting_proteins:
        print(ip)
else:
    print("\nSpecified gene was not found in String database, double check that you have it correctly!")
    interacting_proteins.append(gene)

Accessing String database, response status:  200

Top 30 PIK3CA interacting proteins: 

PIK3R2
TNS1
RPS6KB1
KRAS
PDGFRA
PIK3R3
AKT3
PIK3CA
ERBB3
IGF1R
ERBB2
EGFR
GNAQ
KIT
MRAS
MAP2K1
IRS1
RHOG
FGFR3
PDPK1
CTNNB1
NRAS
PTEN
IRS2
AKT2
CDC42
ESR1
HRAS
CDKN2A
PIK3R1
AKT1


## Show network of interacting proteins 

In [37]:
'''Display image of network of interacting proteins (maybe not worth it?)'''
string_api_url = "https://string-db.org/api"
output_format = "image"
method = "network"

my_genes = [gene]
species = "9606"
request_url = string_api_url + "/" + output_format + "/" + method + "?"
request_url += "identifiers=%s"
request_url += "&" + "species=" + species
request_url += "&" + "add_white_nodes=15" 

try:
    http = urllib3.PoolManager()
    response = http.request('GET',request_url)
    
except urllib3.HTTPError as err:
    error_message = err.read()
    print(error_message)
    sys.exit()
    
print(response.data)

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xcf\x00\x00\x01k\x08\x06\x00\x00\x00\xef\x99\xbb\x12\x00\x00\x00\x04sBIT\x08\x08\x08\x08|\x08d\x88\x00\x00\x00\tpHYs\x00\x00\r\xd7\x00\x00\r\xd7\x01B(\x9bx\x00\x00\x00\x19tEXtSoftware\x00www.inkscape.org\x9b\xee<\x1a\x00\x00 \x00IDATx\x9c\xec\xddy|T\xd5\xfd?\xfe\xd79w\x99-3\xd9C\x16\xb2\x10\xd6\xb0\x83 \x8b,\x82[]p\x87\xbaR\xd4\xfe\xa8\xf8\xa9\x8a[m\xd5O\x8b\xd4\x8f\xd6\xe5c\xad\x1f\xad\xad\xd5*\xb6\x16\x85\xaa\x95\xd6\x15\x05\x17T@\xc0\xa0@\xd8IB\x08!\xfb6\xeb\xbd\xe7\x9c\xdf\x1f37\x0e1(\xa0\xf6[\xe5\xfd|<\xeecf2\x93{\xe7\xde\xe0c|\xcd\xfb\x9c\xf3\x06\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B\x08!\x84\x10B